In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

text =f'sanjeev has good heart'

doc=nlp(text)

for token in doc:
    print(token.text, token.pos_)

In [1]:
import glob
from pdf2image import convert_from_path
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
import pickle
import cv2

In [2]:
def convert_pdf2image(pdf_path, output_path, poppler_path):
    file_path = glob.glob(pdf_path)
    img_list=[]
    for file in file_path:
        img_name= file.split("\\")[1]
        img_name = output_path+img_name.split(".")[0]+'.png'
        images = convert_from_path(file, 300, poppler_path=poppler_path)
        for image in images:
            image.save(img_name, 'PNG')
            img_list.append(images)
    
    return  img_list
    

In [3]:
pdf_path = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/data_to_work/*.pdf"
poppler_path = 'C:/python_lib/poppler-0.68.0/bin'
outfolder = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/output/"
img_file_path = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/output/*.png"
csv_path ='C:/Users/Sanjeev/Documents/Python Scripts/Dataset/csv/'
tesseract_path ='C:/python_lib/ocr/tesseract.exe'
data_path = "C:/Users/Sanjeev/Documents/Python Scripts/Dataset/df_word_list"
pytesseract.pytesseract.tesseract_cmd = 'C:/python_lib/ocr/tesseract.exe'

In [ ]:
img_list = convert_pdf2image(pdf_path,outfolder,poppler_path)

In [4]:

def extract_bbox_info(img_path):
    hocr_data = pytesseract.image_to_pdf_or_hocr(img_path, extension='hocr')
    soup = BeautifulSoup(hocr_data)
    spans = soup.find_all('span', attrs={'class':'ocrx_word'})  # getting the attributes 
    word_bbox_list =[]  
    id=[]
    word=[]
    box =[]
    for span in spans:
        bbox_str=span['title']  # geting bounding box inforamtion 
        bbox_str =bbox_str.split(';')
        bbox =bbox_str[0].split()[1:]
    #     x,y,w,h= *bbox
    #     word_info_list.append([span['id'],span.string,[bbox[1].split()[-1],*bbox]])
        word_bbox_list.append([span['id'],span.string,
                               [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                              ])
        id.append(span['id'])
        word.append(span.string)
        box.append(str([int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]))
#         bbox_width.append((int(bbox[2])-int(bbox[0])))
#         word_info_list.sort(key = lambda sort_value: (sort_value[2][1],sort_value[2][0]))
#     word_info_list.sort(key = lambda sort_value: sort_value[2][1])
    data={'word_id': id,'word':word,'bbox':box,'label':None,'key':None,'value':None}
#     return word_bbox_list, bbox_width, pd.DataFrame(data)
    return word_bbox_list,pd.DataFrame(data)

In [ ]:
img_path  = glob.glob(img_file_path)
df_list =[]
word_list =[]
for img in img_path:
    img_name = img.split("\\")[1]
    img_name = csv_path+img_name.split('.')[0]+'.csv'
    word,df = extract_bbox_info(img)
    df_list.append(df)
    word_list.append(word)
    df.to_csv(img_name)
#     break


In [5]:
def SaveData(data,path):
    with open(path,'wb') as file:
        pickle.dump(data,file)

def LoadData(path):
    with open(path, 'rb') as file:
        return pickle.load(file)
    
def print_ocr_string(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ..\n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
    print(text)  
    
def display_bounding_box(img, word_bbox_list,title="word bounding box"):
#     img = cv2.imread(img)
    for bbox in word_bbox_list:
        b = bbox[-1]
        img = cv2.rectangle(img, (int(b[0]), int(b[1])), (int(b[2]), int(b[3])), (0, 255, 0), 2)

    # show annotated image and wait for keypress
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, img)


In [6]:
# import sys
# sys.setrecursionlimit(10000)
# SaveData([df_list, word_list], data_path)
data= LoadData(data_path)

df_list=data[0]
word_list=data[1]

In [7]:
word_list[0]

[['word_1_1', 'STANDARD', [1425, 74, 1839, 130]],
 ['word_1_2', 'INVOICE', [1869, 74, 2168, 130]],
 ['word_1_3', 'Ai', [162, 113, 236, 194]],
 ['word_1_4', 'gas', [243, 124, 414, 202]],
 ['word_1_5', 'AIRGAS', [472, 131, 596, 155]],
 ['word_1_6', 'USA,', [610, 131, 682, 160]],
 ['word_1_7', 'LLC', [697, 131, 754, 155]],
 ['word_1_8', 'i.', [411, 159, 424, 198]],
 ['word_1_9', 'PO', [475, 165, 519, 189]],
 ['word_1_10', 'Box', [532, 165, 586, 189]],
 ['word_1_11', '9249', [597, 165, 669, 189]],
 ['word_1_12', 'INVOICE', [1151, 163, 1234, 178]],
 ['word_1_13', 'DATE', [1242, 163, 1295, 178]],
 ['word_1_14', 'PAYER', [1413, 163, 1481, 178]],
 ['word_1_15', 'INVOICE', [1629, 163, 1712, 178]],
 ['word_1_16', 'NO.', [1720, 163, 1754, 178]],
 ['word_1_17', 'DUE', [1882, 163, 1924, 178]],
 ['word_1_18', 'DATE', [1932, 163, 1985, 178]],
 ['word_1_19', 'PAY', [2169, 163, 2209, 178]],
 ['word_1_20', 'THIS', [2216, 163, 2262, 178]],
 ['word_1_21', 'AMOUNT', [2269, 163, 2358, 178]],
 ['word_1_22', 

In [8]:
img_path =glob.glob(img_file_path)
print(img_path[0])
print_ocr_string(img_path[0])

C:/Users/Sanjeev/Documents/Python Scripts/Dataset/output\0001723192_9097101023_20200311210717.png
reading image ..

STANDARD INVOICE

Ai rgas AIRGAS USA, LLC
; PO Box 9249 INVOICE DATE PAYER INVOICE NO. DUE DATE PAY THIS AMOUNT
an Air Liquide company Marietta, GA 30065-2249 01/14/2020| 1723192 | 9097101023 | 02/13/2020 $ 56.27

Manage Your Account Online 24/7
sotDBY AIRGAS USA, LLC (S023) Access order history, view cylinder balances, get proofs of delivery,
5125 NW 77TH AVE pay invoices and more -- visit Airgas.com today
MIAMI FL 33166-5524
305-470-8933

PLEASE MAKE CHECKS PAYABLE AND REMIT TO:

BILLTO BOSTON SCIENTIFIC CO CRM LE [ee eg ETT ey
ACCOUNTS PAYABLE DEPARTMENT AIRGAS USA, LLC

PO BOX 9198 PO BOX 734672
CANTON MA 02021 DALLAS TX 75373-4672

1723192190971010230000005be2749

TO ENSURE PROPER CREDIT, PLEASE RETURN THE UPPER PORTION WITH YOUR REMITTANCE. FOR QUESTIONS ON YOUR ACCOUNT PLEASE CALL: 800-727-0693

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

In [11]:
img = cv2.imread(img_path[0])
display_bounding_box(img,word_list[0])